In [1]:
# STEP 1: Generate Mock MCA Data (3 days × 5 states)
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Fix the typo in "Gujarat" from the PDF
states = ["Maharashtra", "Gujarat", "Delhi", "Tamil Nadu", "Karnataka"]
company_classes = ["Private", "Public", "One Person Company"]
statuses = ["Active", "Strike Off", "Amalgamated", "Dissolved"]
nic_codes = {
    "1010": "Manufacturing",
    "6202": "Computer Programming",
    "6820": "Real Estate",
    "8299": "Business Support Services"
}

np.random.seed(42)

def generate_state_data(state, day_offset=0):
    # Base number of companies per state
    base_n = 100
    if day_offset == 0:
        n = base_n
    else:
        # Add 5-10 new companies each day
        n = base_n + np.random.randint(5, 11)

    base_date = datetime(2025, 10, 15) + timedelta(days=day_offset)  # Start from recent date

    data = []
    existing_cins = set()

    for i in range(n):
        # CIN format: L/U + 20 digits (L = Indian, U = Foreign)
        prefix = "L" if state in ["Maharashtra", "Delhi", "Karnataka"] else "U"
        cin = prefix + ''.join(np.random.choice(list('0123456789'), 20))
        while cin in existing_cins:
            cin = prefix + ''.join(np.random.choice(list('0123456789'), 20))
        existing_cins.add(cin)

        # Simulate changes over days
        if day_offset == 0:
            status = np.random.choice(statuses, p=[0.85, 0.10, 0.03, 0.02])
            auth_cap = np.random.choice([100000, 500000, 1000000, 5000000])
        else:
            # Slight drift in status and capital
            status = np.random.choice(statuses, p=[0.87, 0.08, 0.03, 0.02])
            auth_cap = np.random.choice([100000, 500000, 1000000, 5000000, 10000000])

        paid_up = min(auth_cap, np.random.choice([50000, 100000, 200000, 500000]))
        nic = np.random.choice(list(nic_codes.keys()))

        data.append({
            "CIN": cin,
            "Company_Name": f"{state.replace(' ', '')} Innovations Pvt Ltd {i+1}",
            "Company_Class": np.random.choice(company_classes),
            "Date_of_Incorporation": (base_date - timedelta(days=np.random.randint(0, 730))).strftime("%d/%m/%Y"),
            "Authorized_Capital": auth_cap,
            "Paid_up_Capital": paid_up,
            "Company_Status": status,
            "Principal_Business_Activity": nic,
            "Registered_Office_Address": f"Plot {i%50}, {state} Tech Park",
            "ROC_Code": f"ROC-{state[:3].upper()}"
        })

    return pd.DataFrame(data)

# Create directory structure
os.makedirs("mca_data/day1", exist_ok=True)
os.makedirs("mca_data/day2", exist_ok=True)
os.makedirs("mca_data/day3", exist_ok=True)

# Generate data for each state and day
for state in states:
    for day in range(3):
        df = generate_state_data(state, day)
        filename = f"mca_data/day{day+1}/{state.replace(' ', '_')}.csv"
        df.to_csv(filename, index=False)

print("✅ Mock MCA data generated for 5 states across 3 days!")
print("📁 Files saved in: mca_data/day1/, day2/, day3/")

✅ Mock MCA data generated for 5 states across 3 days!
📁 Files saved in: mca_data/day1/, day2/, day3/


In [2]:
df_sample = pd.read_csv("mca_data/day1/Maharashtra.csv")
print("Sample from Day 1 - Maharashtra:")
print(df_sample.head(2))
print("\nShape:", df_sample.shape)

Sample from Day 1 - Maharashtra:
                     CIN                       Company_Name  \
0  L63746926743772541751  Maharashtra Innovations Pvt Ltd 1   
1  L80926382426486138198  Maharashtra Innovations Pvt Ltd 2   

        Company_Class Date_of_Incorporation  Authorized_Capital  \
0              Public            24/09/2025              100000   
1  One Person Company            18/07/2024              500000   

   Paid_up_Capital Company_Status  Principal_Business_Activity  \
0            50000         Active                         8299   
1           500000         Active                         8299   

       Registered_Office_Address ROC_Code  
0  Plot 0, Maharashtra Tech Park  ROC-MAH  
1  Plot 1, Maharashtra Tech Park  ROC-MAH  

Shape: (100, 10)


In [3]:
# STEP 2: Data Integration
import pandas as pd
import os

states = ["Maharashtra", "Gujarat", "Delhi", "Tamil Nadu", "Karnataka"]

def integrate_day_data(day_num):
    print(f"🔄 Integrating Day {day_num} data...")
    all_dfs = []

    for state in states:
        filename = f"mca_data/day{day_num}/{state.replace(' ', '_')}.csv"
        if os.path.exists(filename):
            df = pd.read_csv(filename)
            df["STATE"] = state
            all_dfs.append(df)
        else:
            print(f"⚠️ Warning: {filename} not found!")

    if not all_dfs:
        raise FileNotFoundError(f"No data found for Day {day_num}")

    master = pd.concat(all_dfs, ignore_index=True)

    # 🔧 FIX: Standardize column names SAFELY
    # Keep 'CIN' as-is; only clean other columns
    new_columns = []
    for col in master.columns:
        if col.strip().upper() == "CIN":
            new_columns.append("CIN")  # preserve exact name
        else:
            # Clean others: strip, replace underscores, title case
            clean_col = col.strip().replace("_", " ").title()
            new_columns.append(clean_col)
    master.columns = new_columns

    # Now ensure "CIN" exists
    if "CIN" not in master.columns:
        raise KeyError("CIN column missing after standardization!")

    # Fix data
    master["Company Status"] = master["Company Status"].astype(str).str.strip()
    master["State"] = master["State"].astype(str).str.strip()

    # Handle nulls
    numeric_cols = ["Authorized Capital", "Paid Up Capital"]
    text_cols = ["Company Name", "Registered Office Address", "Company Class"]

    for col in numeric_cols:
        if col in master.columns:
            master[col] = pd.to_numeric(master[col], errors='coerce').fillna(0).astype(int)

    for col in text_cols:
        if col in master.columns:
            master[col] = master[col].fillna("N/A").astype(str)

    # Deduplicate using CIN (now guaranteed to exist)
    master = master.drop_duplicates(subset=["CIN"], keep="first")

    # Save
    output_path = f"master_day{day_num}.csv"
    master.to_csv(output_path, index=False)
    print(f"✅ Day {day_num} master saved: {output_path} | Rows: {len(master)}")
    return master

# Run for all days
master_day1 = integrate_day_data(1)
master_day2 = integrate_day_data(2)
master_day3 = integrate_day_data(3)

print("\n🎉 Data integration complete!")

🔄 Integrating Day 1 data...
✅ Day 1 master saved: master_day1.csv | Rows: 500
🔄 Integrating Day 2 data...
✅ Day 2 master saved: master_day2.csv | Rows: 537
🔄 Integrating Day 3 data...
✅ Day 3 master saved: master_day3.csv | Rows: 540

🎉 Data integration complete!


In [4]:
# STEP 3: Change Detection Engine
import pandas as pd
from datetime import datetime, timedelta

# Load master datasets
master1 = pd.read_csv("master_day1.csv")
master2 = pd.read_csv("master_day2.csv")
master3 = pd.read_csv("master_day3.csv")

# Ensure CIN is string (critical for comparison)
for df in [master1, master2, master3]:
    df["CIN"] = df["CIN"].astype(str)

def detect_changes(old_df, new_df, change_date_str):
    print(f"🔍 Detecting changes up to {change_date_str}...")
    changes = []

    # Convert to dict for fast lookup: {CIN: row}
    old_dict = old_df.set_index("CIN").to_dict(orient="index")
    new_dict = new_df.set_index("CIN").to_dict(orient="index")

    old_cins = set(old_dict.keys())
    new_cins = set(new_dict.keys())

    # 1. New Incorporations
    for cin in (new_cins - old_cins):
        changes.append({
            "CIN": cin,
            "Change_Type": "New Incorporation",
            "Field_Changed": "",
            "Old_Value": "",
            "New_Value": "",
            "Date": change_date_str
        })

    # 2. Deregistrations / Missing CINs
    for cin in (old_cins - new_cins):
        changes.append({
            "CIN": cin,
            "Change_Type": "Deregistered/Struck Off",
            "Field_Changed": "",
            "Old_Value": "",
            "New_Value": "",
            "Date": change_date_str
        })

    # 3. Field Updates (only for CINs present in both)
    common_cins = old_cins & new_cins
    fields_to_check = [
        "Company Status",
        "Authorized Capital",
        "Paid Up Capital",
        "Company Class",
        "Principal Business Activity"
    ]

    for cin in common_cins:
        old_row = old_dict[cin]
        new_row = new_dict[cin]
        for field in fields_to_check:
            if field in old_row and field in new_row:
                old_val = str(old_row[field]).strip()
                new_val = str(new_row[field]).strip()
                if old_val != new_val:
                    changes.append({
                        "CIN": cin,
                        "Change_Type": "Field Update",
                        "Field_Changed": field,
                        "Old_Value": old_val,
                        "New_Value": new_val,
                        "Date": change_date_str
                    })

    return pd.DataFrame(changes)

# Define dates (use realistic dates based on your data)
# From Step 1, we used base date = 2025-10-15 → so:
# Day1 = 2025-10-15, Day2 = 2025-10-16, Day3 = 2025-10-17
date_day2 = "2025-10-16"
date_day3 = "2025-10-17"

# Detect changes
changes_1_to_2 = detect_changes(master1, master2, date_day2)
changes_2_to_3 = detect_changes(master2, master3, date_day3)

# Save change logs
changes_1_to_2.to_csv("changes_day1_to_day2.csv", index=False)
changes_2_to_3.to_csv("changes_day2_to_day3.csv", index=False)

print(f"✅ Changes Day1→Day2: {len(changes_1_to_2)} records saved to changes_day1_to_day2.csv")
print(f"✅ Changes Day2→Day3: {len(changes_2_to_3)} records saved to changes_day2_to_day3.csv")

# Combine all changes for enrichment & dashboard
all_changes = pd.concat([changes_1_to_2, changes_2_to_3], ignore_index=True)
all_changes.to_csv("all_changes.csv", index=False)
print(f"📦 All changes combined: {len(all_changes)} records → all_changes.csv")

🔍 Detecting changes up to 2025-10-16...
🔍 Detecting changes up to 2025-10-17...
✅ Changes Day1→Day2: 1037 records saved to changes_day1_to_day2.csv
✅ Changes Day2→Day3: 1077 records saved to changes_day2_to_day3.csv
📦 All changes combined: 2114 records → all_changes.csv


In [5]:
# STEP 4: Web-Based CIN Enrichment (Mocked Implementation)
import pandas as pd
import random
import os

# Load latest master data (to get COMPANY_NAME, STATE, STATUS, NIC)
master_latest = pd.read_csv("master_day3.csv")
master_latest["CIN"] = master_latest["CIN"].astype(str)

# Load all changes and sample 75 unique CINs
all_changes = pd.read_csv("all_changes.csv")
changed_cins = all_changes["CIN"].drop_duplicates().sample(n=min(75, len(all_changes)), random_state=42).tolist()

# Filter master data for these CINs
enrich_subset = master_latest[master_latest["CIN"].isin(changed_cins)].copy()

# NIC to Sector mapping (from MCA/NIC standard)
nic_to_sector = {
    "1010": "Manufacturing",
    "6202": "Computer Programming & IT Services",
    "6820": "Real Estate Activities",
    "8299": "Business Support Services",
    "4610": "Wholesale Trade",
    "6499": "Financial Services",
    "8530": "Higher Education",
    "9499": "Membership Organizations"
}

# Fake director names (for realism)
fake_directors = [
    "Rajesh Kumar", "Priya Sharma", "Amit Patel", "Sneha Reddy", "Vikram Singh",
    "Anjali Desai", "Karthik Nair", "Meera Iyer", "Sanjay Mehta", "Divya Agarwal"
]

# Build enriched records in required format
enriched_records = []

for _, row in enrich_subset.iterrows():
    cin = row["CIN"]
    company_name = row["Company Name"]
    state = row["State"]
    status = row["Company Status"]
    nic = str(row.get("Principal Business Activity", "8299")).strip()

    # Map NIC to sector (fallback to "Other")
    sector = nic_to_sector.get(nic[:4], "Other Services")

    # Generate fake director
    director = random.choice(fake_directors)

    # Simulated source URLs (ZaubaCorp style)
    zauba_url = f"https://www.zaubacorp.com/company/{cin}"

    # Add multiple enrichment rows per company (as per format)
    enriched_records.extend([
        [cin, company_name, state, status, "ZaubaCorp", "SECTOR", sector],
        [cin, company_name, state, status, "ZaubaCorp", "DIRECTOR", director],
        [cin, company_name, state, status, "ZaubaCorp", "SOURCE_URL", zauba_url]
    ])

# Create final enriched DataFrame
enriched_df = pd.DataFrame(
    enriched_records,
    columns=["CIN", "COMPANY_NAME", "STATE", "STATUS", "SOURCE", "FIELD", "SOURCE_URL"]
)

# Save to CSV
enriched_df.to_csv("enriched_data.csv", index=False)
print(f"✅ Enriched data saved: enriched_data.csv")
print(f"📊 Sample rows:")
print(enriched_df.head(6))

✅ Enriched data saved: enriched_data.csv
📊 Sample rows:
                     CIN                        COMPANY_NAME        STATE  \
0  L94811591354050294949  Maharashtra Innovations Pvt Ltd 74  Maharashtra   
1  L94811591354050294949  Maharashtra Innovations Pvt Ltd 74  Maharashtra   
2  L94811591354050294949  Maharashtra Innovations Pvt Ltd 74  Maharashtra   
3  L04237641559958770465  Maharashtra Innovations Pvt Ltd 75  Maharashtra   
4  L04237641559958770465  Maharashtra Innovations Pvt Ltd 75  Maharashtra   
5  L04237641559958770465  Maharashtra Innovations Pvt Ltd 75  Maharashtra   

   STATUS     SOURCE       FIELD  \
0  Active  ZaubaCorp      SECTOR   
1  Active  ZaubaCorp    DIRECTOR   
2  Active  ZaubaCorp  SOURCE_URL   
3  Active  ZaubaCorp      SECTOR   
4  Active  ZaubaCorp    DIRECTOR   
5  Active  ZaubaCorp  SOURCE_URL   

                                          SOURCE_URL  
0                          Business Support Services  
1                                        

In [6]:
# STEP 5: AI Summary Generator
import pandas as pd
import json

# Load change logs
changes_1_to_2 = pd.read_csv("changes_day1_to_day2.csv")
changes_2_to_3 = pd.read_csv("changes_day2_to_day3.csv")

def generate_summary(df, date_str):
    # Count change types
    new_incorp = int((df["Change_Type"] == "New Incorporation").sum())
    deregistered = int((df["Change_Type"] == "Deregistered/Struck Off").sum())
    field_updates = int((df["Change_Type"] == "Field Update").sum())

    summary_text = f"Daily Summary\nNew incorporations: {new_incorp}\nDeregistered: {deregistered}\nUpdated records: {field_updates}"

    return {
        "date": date_str,
        "new_incorporations": new_incorp,
        "deregistered": deregistered,
        "field_updates": field_updates,
        "summary": summary_text
    }

# Generate summaries
summary_day2 = generate_summary(changes_1_to_2, "2025-10-16")
summary_day3 = generate_summary(changes_2_to_3, "2025-10-17")

# Combine into one JSON structure
daily_summaries = {
    "summaries": [summary_day2, summary_day3]
}

# Save to JSON
with open("daily_summary.json", "w") as f:
    json.dump(daily_summaries, f, indent=2)

# Also save a .txt version (as shown in PDF)
with open("daily_summary.txt", "w") as f:
    f.write(summary_day3["summary"])  # Latest day as example

print("✅ AI Summaries generated!")
print("\n📄 Latest Summary (2025-10-17):")
print(summary_day3["summary"])
print("\n💾 Saved as: daily_summary.json and daily_summary.txt")

✅ AI Summaries generated!

📄 Latest Summary (2025-10-17):
Daily Summary
New incorporations: 540
Deregistered: 537
Updated records: 0

💾 Saved as: daily_summary.json and daily_summary.txt


In [7]:
# STEP 6: Conversational Chatbot (Rule-Based)
import pandas as pd
import re

# Load latest data
master_df = pd.read_csv("master_day3.csv")
master_df["CIN"] = master_df["CIN"].astype(str)
master_df["Authorized Capital"] = pd.to_numeric(master_df["Authorized Capital"], errors='coerce')

all_changes = pd.read_csv("all_changes.csv")
all_changes["CIN"] = all_changes["CIN"].astype(str)

def chatbot_response(query):
    query = query.lower().strip()
    print(f"\n💬 You asked: '{query}'")

    # 1. New incorporations in [STATE]
    if "new incorporation" in query or ("new" in query and "incorporation" in query):
        if "maharashtra" in query:
            state_filter = "Maharashtra"
        elif "gujarat" in query:
            state_filter = "Gujarat"
        elif "delhi" in query:
            state_filter = "Delhi"
        elif "tamil" in query or "nadu" in query:
            state_filter = "Tamil Nadu"
        elif "karnataka" in query:
            state_filter = "Karnataka"
        else:
            state_filter = None

        new_cins = all_changes[all_changes["Change_Type"] == "New Incorporation"]["CIN"].tolist()
        if state_filter:
            filtered = master_df[
                (master_df["CIN"].isin(new_cins)) &
                (master_df["State"] == state_filter)
            ][["CIN", "Company Name"]]
            if not filtered.empty:
                result = f"✅ Found {len(filtered)} new incorporations in {state_filter}:\n"
                for _, row in filtered.head(5).iterrows():
                    result += f"  • {row['Company Name']} ({row['CIN']})\n"
                return result
            else:
                return f"❌ No new incorporations found in {state_filter}."
        else:
            count = len(new_cins)
            return f"📊 Total new incorporations: {count}"

    # 2. Struck off / Deregistered count
    elif "struck off" in query or "deregister" in query or "strike off" in query:
        count = len(all_changes[all_changes["Change_Type"] == "Deregistered/Struck Off"])
        return f"🗑️ Companies struck off/deregistered: {count}"

    # 3. Companies with capital above X
    elif "capital" in query and ("above" in query or "greater" in query):
        # Extract number (e.g., "10 lakh" → 1000000)
        lakh_match = re.search(r'(\d+)\s*lakh', query)
        if lakh_match:
            lakh_val = int(lakh_match.group(1))
            min_cap = lakh_val * 100000  # 1 lakh = 100,000
        else:
            # Try raw number (assume it's in INR)
            num_match = re.search(r'(\d{5,})', query)
            if num_match:
                min_cap = int(num_match.group(1))
            else:
                min_cap = 1000000  # default: 10 lakh

        high_cap = master_df[master_df["Authorized Capital"] > min_cap][["Company Name", "Authorized Capital"]]
        if not high_cap.empty:
            result = f"💰 Found {len(high_cap)} companies with capital > ₹{min_cap:,}:\n"
            for _, row in high_cap.head(5).iterrows():
                result += f"  • {row['Company Name']} (₹{int(row['Authorized Capital']):,})\n"
            return result
        else:
            return f"❌ No companies found with capital > ₹{min_cap:,}."

    # 4. Default fallback
    else:
        return (
            "🤖 I understand queries like:\n"
            "• 'Show new incorporations in Maharashtra'\n"
            "• 'How many companies were struck off?'\n"
            "• 'List companies with capital above 10 lakh'\n"
            "Try one of these!"
        )

# 🔍 Test the chatbot with example queries
test_queries = [
    "Show new incorporations in Maharashtra.",
    "How many companies were struck off?",
    "List companies with capital above 10 lakh."
]

for q in test_queries:
    response = chatbot_response(q)
    print(response)
    print("-" * 50)


💬 You asked: 'show new incorporations in maharashtra.'
✅ Found 106 new incorporations in Maharashtra:
  • Maharashtra Innovations Pvt Ltd 1 (L57533537803315264347)
  • Maharashtra Innovations Pvt Ltd 2 (L60851436704202567324)
  • Maharashtra Innovations Pvt Ltd 3 (L87603665759232094001)
  • Maharashtra Innovations Pvt Ltd 4 (L24338632525853452737)
  • Maharashtra Innovations Pvt Ltd 5 (L83726269951253509027)

--------------------------------------------------

💬 You asked: 'how many companies were struck off?'
🗑️ Companies struck off/deregistered: 1037
--------------------------------------------------

💬 You asked: 'list companies with capital above 10 lakh.'
💰 Found 232 companies with capital > ₹1,000,000:
  • Maharashtra Innovations Pvt Ltd 1 (₹5,000,000)
  • Maharashtra Innovations Pvt Ltd 3 (₹10,000,000)
  • Maharashtra Innovations Pvt Ltd 5 (₹5,000,000)
  • Maharashtra Innovations Pvt Ltd 9 (₹10,000,000)
  • Maharashtra Innovations Pvt Ltd 10 (₹10,000,000)

---------------------

In [8]:
# STEP 7: Streamlit Dashboard — Save as app.py
%%writefile app.py

import streamlit as st
import pandas as pd
import json
import re
from datetime import datetime

# ----------------------------
# Load Data
# ----------------------------
@st.cache_data
def load_data():
    master = pd.read_csv("master_day3.csv")
    changes = pd.read_csv("all_changes.csv")
    enriched = pd.read_csv("enriched_data.csv")
    with open("daily_summary.json", "r") as f:
        summaries = json.load(f)
    return master, changes, enriched, summaries

master_df, all_changes, enriched_df, daily_summaries = load_data()

# Ensure CIN is string
master_df["CIN"] = master_df["CIN"].astype(str)
all_changes["CIN"] = all_changes["CIN"].astype(str)

# ----------------------------
# Chatbot Logic (from Step 6)
# ----------------------------
def chatbot_response(query):
    query = query.lower().strip()
    if "new incorporation" in query or ("new" in query and "incorporation" in query):
        state_map = {
            "maharashtra": "Maharashtra",
            "gujarat": "Gujarat",
            "delhi": "Delhi",
            "tamil": "Tamil Nadu",
            "nadu": "Tamil Nadu",
            "karnataka": "Karnataka"
        }
        state_filter = None
        for kw, state in state_map.items():
            if kw in query:
                state_filter = state
                break

        new_cins = all_changes[all_changes["Change_Type"] == "New Incorporation"]["CIN"].tolist()
        if state_filter:
            filtered = master_df[
                (master_df["CIN"].isin(new_cins)) &
                (master_df["State"] == state_filter)
            ][["CIN", "Company Name"]]
            if not filtered.empty:
                result = f"✅ Found {len(filtered)} new incorporations in {state_filter}:\n\n"
                for _, row in filtered.head(5).iterrows():
                    result += f"- **{row['Company Name']}** (`{row['CIN']}`)\n"
                return result
            else:
                return f"❌ No new incorporations found in {state_filter}."
        else:
            count = len(new_cins)
            return f"📊 Total new incorporations: **{count}**"

    elif "struck off" in query or "deregister" in query or "strike off" in query:
        count = len(all_changes[all_changes["Change_Type"] == "Deregistered/Struck Off"])
        return f"🗑️ Companies struck off/deregistered: **{count}**"

    elif "capital" in query and ("above" in query or "greater" in query):
        lakh_match = re.search(r'(\d+)\s*lakh', query)
        if lakh_match:
            min_cap = int(lakh_match.group(1)) * 100000
        else:
            num_match = re.search(r'(\d{5,})', query)
            min_cap = int(num_match.group(1)) if num_match else 1000000

        high_cap = master_df[master_df["Authorized Capital"] > min_cap][["Company Name", "Authorized Capital"]]
        if not high_cap.empty:
            result = f"💰 Found {len(high_cap)} companies with capital > ₹{min_cap:,}:\n\n"
            for _, row in high_cap.head(5).iterrows():
                result += f"- **{row['Company Name']}** (₹{int(row['Authorized Capital']):,})\n"
            return result
        else:
            return f"❌ No companies found with capital > ₹{min_cap:,}."

    else:
        return (
            "🤖 Try these examples:\n\n"
            "- *Show new incorporations in Maharashtra*\n"
            "- *How many companies were struck off?*\n"
            "- *List companies with capital above 10 lakh*"
        )

# ----------------------------
# Streamlit App
# ----------------------------
st.set_page_config(page_title="MCA Insights Engine", layout="wide")
st.title("🔍 MCA Insights Engine")
st.markdown("Track company changes across Maharashtra, Gujarat, Delhi, Tamil Nadu & Karnataka")

# Sidebar Filters
st.sidebar.header("Filters")
states = ["All"] + sorted(master_df["State"].dropna().unique().tolist())
selected_state = st.sidebar.selectbox("State", states)
statuses = ["All"] + sorted(master_df["Company Status"].dropna().unique().tolist())
selected_status = st.sidebar.selectbox("Company Status", statuses)

# Search
search_query = st.text_input("🔍 Search by CIN or Company Name")

# Filter data
filtered_df = master_df.copy()
if selected_state != "All":
    filtered_df = filtered_df[filtered_df["State"] == selected_state]
if selected_status != "All":
    filtered_df = filtered_df[filtered_df["Company Status"] == selected_status]
if search_query:
    filtered_df = filtered_df[
        filtered_df["CIN"].str.contains(search_query, case=False, na=False) |
        filtered_df["Company Name"].str.contains(search_query, case=False, na=False)
    ]

# Tabs
tab1, tab2, tab3, tab4 = st.tabs(["📊 Dashboard", "📈 Change History", "🧩 Enriched Data", "💬 Chat with Data"])

# Tab 1: Dashboard
with tab1:
    st.subheader("Company Records")
    st.dataframe(filtered_df[[
        "CIN", "Company Name", "State", "Company Status",
        "Authorized Capital", "Paid Up Capital", "Principal Business Activity"
    ]].head(20), use_container_width=True)

# Tab 2: Change History
with tab2:
    st.subheader("Daily Change Trends")
    change_counts = all_changes.groupby(["Date", "Change_Type"]).size().unstack(fill_value=0)
    st.bar_chart(change_counts)

# Tab 3: Enriched Data
with tab3:
    st.subheader("Enriched Company Info (Sample)")
    enriched_sample = enriched_df.head(15)
    st.dataframe(enriched_sample, use_container_width=True)

# Tab 4: Chatbot
with tab4:
    st.subheader("Ask Questions in Natural Language")
    user_input = st.text_input("💬 Type your query below:")
    if user_input:
        response = chatbot_response(user_input)
        st.markdown(response)

# Show latest AI Summary
latest_summary = daily_summaries["summaries"][-1]["summary"]
st.sidebar.markdown("### 📰 Latest AI Summary")
st.sidebar.text(latest_summary)

Writing app.py


In [11]:
from pyngrok import conf
conf.get_default().auth_token = "34IJN7cO4lzAz9UnrJudvvIY0NJ_2cXe5jecf1AD3K7itViyH"  # ← paste your token

In [13]:
# Corrected Colab cell to launch Streamlit + ngrok
!pip install streamlit pyngrok -q
!streamlit run app.py &>/dev/null &
from pyngrok import ngrok

# Optional: If you HAVE an ngrok authtoken, uncomment and add it:
# ngrok.set_auth_token("your_token_here")

public_url = ngrok.connect(8501)
print(f"🔗 Dashboard URL: {public_url}")

🔗 Dashboard URL: NgrokTunnel: "https://tiffiny-unshunnable-jorge.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
https://tiffiny-unshunnable-jorge.ngrok-free.dev/